<a href="https://colab.research.google.com/github/Kebrenok/Simulative-/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A2%D1%80%D0%B5%D1%82%D0%B8%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [19]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-07-07 09:51:02--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.116.4
Connecting to gist.github.com (gist.github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-07-07 09:51:02--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.1’

registrations.csv.1 100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-07-07 09:51:02 (87.2

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Ваше решение
import csv
import math
from datetime import datetime, timedelta

dict_jan = {}
dict_ret = {}

with open('registrations.csv', 'r', newline='') as reg:
  reader=csv.reader(reg, delimiter=';')
  next(reader) #пропускаем заголовок
  for row in reader:
    date_object = datetime.strptime(row[1], "%Y-%m-%d") # приводим строку с датой к нужному формату
    month_number = date_object.month # вытаскиваем из даты месяц и сравниваем с нужным
    if month_number == 1:
      dict_jan[row[0]]=row[1] # словарь январских регистраций (ключ - ID, значение - дата регистрации)
with open('entries.csv', 'r', newline='') as ent:
  reader=csv.reader(ent, delimiter=';')
  next(reader)
  for row in reader:
    # выбираем из файла с заходами те из них, которые совершались на 15 день, при этом запись в словарь исключает дублирование
    if row[0] in dict_jan.keys() and datetime.strptime(row[1], "%Y-%m-%d") == datetime.strptime(dict_jan[row[0]], "%Y-%m-%d") + timedelta(days=15):
      dict_ret[row[0]] = 15 #словарь с ID, у которых был 15-днейный ретеншн
retention_15_day = round(((len(dict_ret) / len(dict_jan))*100),5) #результат
round(retention_15_day, 5)










54.65116

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Ваше решение

import csv
import math
from datetime import datetime, timedelta

dict_jan = {}
dict_ent = {}
dict_ret = {}

with open('registrations.csv', 'r', newline='') as reg:
  reader=csv.reader(reg, delimiter=';')
  next(reader) #пропускаем заголовок
  for row in reader:
    date_object = datetime.strptime(row[1], "%Y-%m-%d") # приводим строку с датой к нужному формату
    month_number = date_object.month # вытаскиваем из даты месяц и сравниваем с нужным
    if month_number == 1:
      dict_jan[row[0]]=date_object # словарь январских регистраций (ключ - ID, значение - дата регистрации)


# Проверяем ежедневные вхождения
with open('entries.csv', 'r', newline='') as ent:
  reader = csv.reader(ent, delimiter=';')
  next(reader)
  user_active_days = {user_id: set() for user_id in dict_jan.keys()}
# наполняем словарь значениями без дубликатов
  for row in reader:
    user_id = row[0]
    entry_date = datetime.strptime(row[1], "%Y-%m-%d")
    if user_id in dict_jan.keys() and entry_date not in user_active_days[user_id]:
      user_active_days[user_id].add(entry_date)

  rolling_retention_rates = set()
#считаем кол-во дней с регистрации:
  for user_reg, reg_day in dict_jan.items():
    for user_id, active_days in user_active_days.items():
      if user_reg==user_id:
        for i in active_days:
          cnt_day = i - reg_day
          if cnt_day.days>=30:
            rolling_retention_rates.add(user_id)
            break
# считаем rolling retention
rolling_retention = round(len(rolling_retention_rates)/len(dict_jan) * 100,5)

round(rolling_retention, 5)

29.06977

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [13]:
# Ваше решение
import csv
import math
from datetime import datetime, timedelta

dict_registrations = {} #словарь регистраций
dict_ret = {}

with open('registrations.csv', 'r', newline='') as reg:
  reader=csv.reader(reg, delimiter=';')
  next(reader) #пропускаем заголовок
  for row in reader:
   dict_registrations[row[0]]=row[1] # словарь регистраций (ключ - ID, значение - дата регистрации)

# Делаем словарь ежедневных вхождений
with open('entries.csv', 'r', newline='') as ent:
  reader = csv.reader(ent, delimiter=';')
  next(reader)
  user_active_days = {user_id: set() for user_id in dict_registrations.keys()}
# наполняем словарь значениями без дубликатов
  for row in reader:
    user_id = row[0]
    entry_date = row[1]
    if entry_date not in user_active_days[user_id]:
      user_active_days[user_id].add(entry_date)

# считаем количество дней с регистрации для каждого входа каждого пользователя

days_since_registration_all = []
for user1, reg_day in dict_registrations.items():
  registration_date = datetime.strptime(reg_day, "%Y-%m-%d")
  for user2, active_days in user_active_days.items():
    if user1==user2:
      user_id = user1
      for ad in active_days:
        entry_date = datetime.strptime(ad, "%Y-%m-%d")
        days = (entry_date - registration_date).days
        days_since_registration_all.append({"user_id": user_id, "days_since_registration": days})

counts = {}
for entry in days_since_registration_all:
    days_since_registration = entry['days_since_registration']
    if days_since_registration not in counts:
        counts[days_since_registration] = 0
    counts[days_since_registration] += 1
each_day_ret = dict(sorted(counts.items()))

day_zero_count = each_day_ret[0]
lifetime = 0

for day, count in each_day_ret.items():
    lifetime += (count / day_zero_count)

lifetime = round(lifetime, 5)


In [14]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [24]:
# Ваше решение

import csv
import math
from datetime import datetime, timedelta

dict_reg = {}
dict_ent = {}
dict_ret = {}

with open('registrations.csv', 'r', newline='') as reg:
  reader=csv.reader(reg, delimiter=';')
  next(reader) #пропускаем заголовок
  for row in reader:
    date_object = datetime.strptime(row[1], "%Y-%m-%d") # приводим строку с датой к нужному формату
    dict_reg[row[0]]=date_object # словарь регистраций (ключ - ID, значение - дата регистрации)


# Проверяем ежедневные вхождения
with open('entries.csv', 'r', newline='') as ent:
  reader = csv.reader(ent, delimiter=';')
  next(reader)
  user_active_days = {user_id: set() for user_id in dict_reg.keys()}
# наполняем словарь значениями без дубликатов
  for row in reader:
    user_id = row[0]
    entry_date = datetime.strptime(row[1], "%Y-%m-%d")
    if user_id in dict_reg.keys() and entry_date not in user_active_days[user_id]:
      user_active_days[user_id].add(entry_date)

  rolling_retention_rates = set()
#считаем кол-во дней с регистрации:
  for user_reg, reg_day in dict_reg.items():
    for user_id, active_days in user_active_days.items():
      if user_reg==user_id:
        for i in active_days:
          cnt_day = i - reg_day
          if cnt_day.days>=29:
            rolling_retention_rates.add(user_id)
            break
# считаем rolling retention
rolling_retention = round(len(rolling_retention_rates)/len(dict_reg),5)


churn_29 = 1-round(rolling_retention, 5)
print(churn_29)

0.509


In [26]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [33]:
# Ваше решение
dec21_entries = set()
for user_id, active_days in user_active_days.items():
  for i in active_days:
    month_number = i.month
    year_number=i.year
    if month_number == 12 and year_number == 2021:
      dec21_entries.add(user_id)

dec_mau = len(dec21_entries)



In [31]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

In [39]:
last_week_dec21_entries = set()
for user_id, active_days in user_active_days.items():
  for i in active_days:
    month_number = i.month
    year_number=i.year
    day_number=i.day
    if month_number == 12 and year_number == 2021 and day_number >= 25 and day_number <= 31:
      last_week_dec21_entries.add(user_id)

dec_wau = len(last_week_dec21_entries)
print(dec_wau)

84


In [40]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

In [44]:
# Ваше решение

last_week_dec21_entries = set()
for user_id, active_days in user_active_days.items():
  for i in active_days:
    month_number = i.month
    year_number=i.year
    day_number=i.day
    if month_number == 12 and year_number == 2021 and day_number == 31:
      last_week_dec21_entries.add(user_id)

dec_dau = len(last_week_dec21_entries)



In [45]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [49]:
# Ваше решение
from collections import defaultdict

# Создаем defaultdict для хранения уникальных юзеров каждого месяца
counts_avg = defaultdict(set)

# Пройдем по входам
for user_id, active_days in user_active_days.items():
  for i in active_days:
    month_number = i.month
    year_number=i.year
    # Используем год и месяц в качестве ключа
    key = f"{year_number}-{month_number}"
    counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts_avg.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_mau = round(sum/len(count_of_unique_user_ids), 5)



102.58333


In [50]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined

In [52]:
# Ваше решение
# Ваше решение
from collections import defaultdict

# Создаем defaultdict для хранения уникальных юзеров каждого месяца
counts_avg = defaultdict(set)

# Пройдем по входам
for user_id, active_days in user_active_days.items():
  for i in active_days:
    year, week_num, _ = i.isocalendar()
    # Используем год и месяц в качестве ключа
    key = f"{year}-{week_num}"
    counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts_avg.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_wau = round(sum/len(count_of_unique_user_ids), 5)



89.86792


In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

In [53]:
# Ваше решение
from collections import defaultdict

# Создаем defaultdict для хранения уникальных юзеров каждого месяца
counts_avg = defaultdict(set)

# Пройдем по входам
for user_id, active_days in user_active_days.items():
  for i in active_days:
    # Используем дату в качестве ключа
    key = i
    counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts_avg.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_dau = round(sum/len(count_of_unique_user_ids), 5)



40.5589


In [54]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'answers' is not defined